In [1]:
import sys
sys.path.append('../config') 

from config import REDDIT_CL_ID, REDDIT_CL_SECRET

import praw
import pandas as pd

In [2]:
user_agent = "scrapper 1.0"
# Credenciales de la aplicación
client_id = REDDIT_CL_ID
client_secret = REDDIT_CL_SECRET

# Crear una instancia de Reddit
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

In [3]:
df_publicaciones = pd.DataFrame(columns=['id', 'titulo', 'autor', 'fecha_submission'])
df_comentarios = pd.DataFrame(columns=['id_publicacion', 'usuario', 'fecha', 'comentario'])

In [4]:
for submission in reddit.subreddit('Twitch').hot(limit=100):
    submission_id = submission.id
    title = submission.title
    author = submission.author
    submission_date = submission.created_utc
    url = submission.url

    # Guardar los detalles de la publicación en el dataframe correspondiente
    df_nueva_publicacion = pd.DataFrame({'id': [submission_id],
                                        'titulo': [title],
                                        'autor': [author],
                                        'fecha_submission': [submission_date],
                                        'url': [url]})
    df_publicaciones = pd.concat([df_publicaciones, df_nueva_publicacion], ignore_index=True)

    # Reemplazar los objetos MoreComments con los comentarios reales
    submission.comments.replace_more(limit=None)
    comments = submission.comments.list()

    # Recorrer los comentarios de la publicación
    for comment in comments:
        comentario_submission_id = submission_id
        usuario = comment.author.name if comment.author else "N/A"
        fecha = comment.created_utc
        comentario = comment.body
        
        # Guardar los comentarios en el dataframe correspondiente
        df_nuevo_comentario = pd.DataFrame({'id_publicacion': [comentario_submission_id],
                                            'usuario': [usuario],
                                            'fecha': [fecha],
                                            'comentario': [comentario]})
        df_comentarios = pd.concat([df_comentarios, df_nuevo_comentario], ignore_index=True)


In [5]:
df_publicaciones

,id,titulo,autor,fecha_submission,url
0,hi9i18,!!Read Before Posting!!,Havryl,1.593467e+09,https://www.reddit.com/r/Twitch/comments/hi9i1...
1,1b3xsrw,Stream Experiences & Stories,AutoModerator,1.709309e+09,https://www.reddit.com/r/Twitch/comments/1b3xs...
2,1b7ewfh,Considering changing my stream to 18+ even tho...,YuSira,1.709669e+09,https://www.reddit.com/r/Twitch/comments/1b7ew...
3,1b83q9k,2 people whispered to me but i can't reply bec...,lolige_eenhoorn,1.709742e+09,https://www.reddit.com/gallery/1b83q9k
4,1b7tnuz,Do you like watching Early streams or Late str...,MarcusBofan,1.709709e+09,https://www.reddit.com/r/Twitch/comments/1b7tn...
...,...,...,...,...,...
95,1b6kmgj,using my elgato capture card to head audio inc...,roospoos,1.709583e+09,https://www.reddit.com/r/Twitch/comments/1b6km...
96,1b6bk2p,Promoting art work spammers appearing more fre...,GeminiNicki,1.709561e+09,https://www.reddit.com/r/Twitch/comments/1b6bk...
97,1b6oxfz,Anyone know that one drum beat people use for ...,AngelLive3,1.709593e+09,https://www.reddit.com/r/Twitch/comments/1b6ox...
98,1b6flhy,Unique On-Screen Twitch Chat,MintMurderMedia,1.709571e+09,https://www.reddit.com/r/Twitch/comments/1b6fl...


In [6]:
df_comentarios

,id_publicacion,usuario,fecha,comentario
0,1b3xsrw,its_Dusty98,1.709630e+09,I just finished a 3-hour live stream of Lego f...
1,1b3xsrw,Puzzleheaded_Bad_331,1.709458e+09,"Hey everyone,I'm back with an update on my jou..."
2,1b7ewfh,leggup,1.709670e+09,My twitch is 13+ (all ages) but my discord is ...
3,1b7ewfh,realcactusart,1.709670e+09,You need to do what you think is right! I have...
4,1b7ewfh,LuminaChannel,1.709671e+09,Do it.\n\n\n Im also a vtuber and I made it cl...
...,...,...,...,...
585,1b6flhy,LaosProdigy,1.709574e+09,From my experience I've found that creating my...
586,1b6flhy,IBlank7,1.709571e+09,A lot of tutorials on google/yt. Just look up ...
587,1b6flhy,MintMurderMedia,1.709592e+09,[https://imgur.com/a/EWyNO60](https://imgur.co...
588,1b6oj4m,WheresWagner,1.709613e+09,Headset will get plugged into the GoXLR
